# Tugas Praktikum Minggu 8
### Nicholas Juan Kalvin P. | 162012133068
<hr>



# Text Classification

## Tugas Praktikum:
<ol>
    <li> Download data teks dari halaman berikut: https://raw.githubusercontent.com/ruzcmc/ClickbaitIndo-textclassifier/master/all_agree.csv</li>
    <li>Lakukan classification menggunakan metode MultinomialNB, RandomForest, dan metode klasifikasi pilihan kalian (terserah) dengan menggunakan fitur TF-IDF </li>
    <li> Buat  perbandingan  performa  ketiga  metode  supervised  learning  tersebut  berdasarkan performanya (poin plus: lakukan cross validation)</li>
</ol>

<hr>


## Requirements

In [1]:
# Module imports
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras.datasets import fashion_mnist
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from pyarrow import csv
import requests
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler 
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline



Pada praktikum kali ini, digunakan 4 library penting untuk melakukan mayoritas dari praktikum. 
<ul>
    <li><code>cv2</code> atau <code>pip:popencv-python</code></li> adalah library yang sangat populer untuk visualisasi, dari visualisasi menggunakan kamera hingga visualisasi gambar digital. Kali ini cv2 digunakan untuk membedah gambar ke beberapa channel untuk tujuan perhitungan.
    <li><code>sklearn</code> adalah library utama dari praktikum ini, sklearn melayani berbagai fungsi statistika hingga pemodelan. Dimasukkan fungsi PCA, StandardScaler, Kmeans dan make_blobs yang bertujuan untuk analisis, prosesing hingga pemodelan data</li>
    <li><code>matplotlib</code> adalah library populer untuk melakukan visualisasi data menjadi grafik</li>
</ul>





<hr>

## Data Loading

### Loading Step 1: Get from GitHub

In [2]:
data_link = 'https://raw.githubusercontent.com/ruzcmc/ClickbaitIndo-textclassifier/master/all_agree.csv'
r = requests.get(data_link)
content = r.content.decode('utf-8')
with open("all_agree.csv", "a") as f:
    f.write(content)
    f.close()

### Loading Step 2: Load as DataFrame

In [3]:
read_options = csv.ReadOptions(use_threads=True, encoding='unicode_escape')
padf = csv.read_csv('all_agree.csv', read_options=read_options)
df = padf.to_pandas()

<hr>

## Data Cleaning

In [4]:
print(df.columns)
print(df.describe())
print(df.isna())
df = df.drop(columns='label')
df.head(5)

Index(['title', 'label', 'label_score'], dtype='object')
                                                    title          label  \
count                                              163665         163665   
unique                                               8603              3   
top     Diawasi di Hong Kong, Aktivis Joshua Wong Terb...  non-clickbait   
freq                                                   57         100643   

       label_score  
count       163665  
unique           3  
top              0  
freq        100643  
        title  label  label_score
0       False  False        False
1       False  False        False
2       False  False        False
3       False  False        False
4       False  False        False
...       ...    ...          ...
163660  False  False        False
163661  False  False        False
163662  False  False        False
163663  False  False        False
163664  False  False        False

[163665 rows x 3 columns]


,title,label_score
0,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be...",0
1,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...,0
2,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,1
3,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh...",0
4,MPR: Amandemen UUD 1945 Tak Akan Melebar ke Ma...,0


In [5]:
df.describe()

,title,label_score
count,163665,163665
unique,8603,3
top,"Diawasi di Hong Kong, Aktivis Joshua Wong Terb...",0
freq,57,100643


In [6]:
df.duplicated().sum()

155061

In [7]:
print(df.duplicated().value_counts())

True     155061
False      8604
dtype: int64


In [8]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [9]:
df['label_score'].value_counts()


0              5289
1              3314
label_score       1
Name: label_score, dtype: int64

<hr>

## Text Cleaning

### Text Cleaning Step 1: Cleaner Functions

In [10]:
import re, string

def bersih_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub("@[A-Za-z0-9_]+", "", text)  # Remove Twitter Mentions
    text = re.sub("#\w+", "", text)
    text = re.sub("\[.*?\]", "", text)
    text = re.sub("https?://\S+|www\.\S+", "", text)
    text = re.sub("<.*?>+", "", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub('\n', '', text)
    text = re.sub("\w*\d\w*", "", text)
    text = re.sub("\d+", "", text)
    text = re.sub("\s+", " ", text).strip()
    text = " ".join(text.split())
    return text


## Text Cleaning Step 2: Running Functions

In [11]:
df['title'] = [bersih_text(t) for t in df['title']]
df

,title,label_score
0,masuk radar pilwalkot medan menantu jokowi ber...,0
1,malaysia sudutkan ri isu kabut asap hingga inv...,0
2,viral driver ojol di bekasi antar pesanan maka...,1
3,kemensos salurkan rp m bagi korban kerusuhan s...,0
4,mpr amandemen uud tak akan melebar ke manamana,0
...,...,...
8609,asap karhutla riau mulai merambah ke nias bmkg...,0
8610,tolak ruu pertanahan ribuan petani siap gelar ...,0
8611,ada niat tambah momongan tanpa ikut program ha...,1
8612,txt akan segera comeback soobin akui gatal ing...,1


<hr>

## Text Preprocessing

### Text Processing step 1: Remove Stopwords

In [12]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

df['title'] = [stopword.remove(kalimat) for i, kalimat in enumerate (df['title'])]
df['title']

0       radar pilwalkot medan menantu jokowi bertemu d...
1         malaysia sudutkan ri isu kabut asap invasi babi
2       viral driver ojol bekasi pesanan makanan pakai...
3       kemensos salurkan rp korban kerusuhan sosial p...
4                      mpr amandemen uud melebar manamana
                              ...                        
8609    asap karhutla riau merambah nias bmkg imbau wa...
8610    tolak ruu pertanahan ribuan petani gelar aksi ...
8611    niat momongan program hamil fedi nuril pengin aja
8612         txt comeback soobin akui gatal pamer spoiler
8613                                                title
Name: title, Length: 8604, dtype: object

In [52]:
df.drop([8613], inplace=True)

### Text Processing Step 2: Count Vectorizing 

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
countvectorizer = CountVectorizer(min_df=5, max_df=0.95)
countvec_title = countvectorizer.fit_transform(df['title'])

In [54]:

countvec = dict(zip(countvectorizer.get_feature_names(), countvec_title.toarray()[0]))
df_cv = pd.DataFrame.from_dict(countvec, columns=['frequency'], orient='index')
df_cv.sort_values(by=['frequency'], ascending = False)

c:\Users\nicho\PycharmProjects\DataMining2\.dm2\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,frequency
medan,1
jokowi,1
nasdem,1
sumut,1
bertemu,1
...,...
inspirasi,0
instagram,0
inter,0
internasional,0


### Text Processing Step 3: TFID Transformation

In [55]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True, sublinear_tf=True) 
X = tfidf_transformer.fit_transform(countvec_title).toarray()
y = df['label_score']

tfidf_df = pd.DataFrame(tfidf_transformer.idf_, index=countvectorizer.get_feature_names(),columns=["idf_weights"]) 
tfidf_df.sort_values(by=['idf_weights'])

c:\Users\nicho\PycharmProjects\DataMining2\.dm2\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,idf_weights
kpk,3.989245
indonesia,4.130393
habibie,4.320190
polisi,4.526593
jokowi,4.534530
...,...
musisi,8.268223
musnahkan,8.268223
nadal,8.268223
pedangdut,8.268223


In [56]:
X.shape

(8603, 2934)

In [58]:
y.shape

(8603,)

<hr>

## Machine Learning

### Machine Learning Step 1: Balancing

In [109]:
smote = SMOTE()
X_bal, y_bal = smote.fit_resample(X, y)

### Machine Learning Step 2: Splitting

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.33, random_state=42)

from sklearn.model_selection import StratifiedKFold

In [113]:
print("Size of train X:", X_train.shape)
print("Size of train y:", y_train.shape)
print("Size of test X:", X_test.shape)
print("Size of test y:", y_test.shape)

Size of train X: (7087, 2934)
Size of train y: (7087,)
Size of test X: (3491, 2934)
Size of test y: (3491,)


### Machine Learning step 3: Modelling

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRFClassifier

random_forest_param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']}

multinomialnb_param_grid = {  
    'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}  

decision_tree_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 10),
    'min_samples_split': range(1, 10),
    'min_samples_leaf': range(1, 5)}
    
sgdclassifier_param_grid = {
    'loss':["hinge", "log_loss", "log", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"],
    'penalty': ["l2", "l1", "elasticnet"]}


rf = GridSearchCV(cv=3, estimator=RandomForestClassifier(), param_grid=random_forest_param_grid)
mnb = GridSearchCV(cv=3, estimator=MultinomialNB(), param_grid=multinomialnb_param_grid)
dt = GridSearchCV(cv=3, estimator=DecisionTreeClassifier(), param_grid=decision_tree_param_grid)
sgd = GridSearchCV(cv=3, estimator=SGDClassifier(), param_grid=sgdclassifier_param_grid)

abc = AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=3)
xgb = XGBRFClassifier(n_estimators=100, subsample=0.9, colsample_bynode=0.2)



### Machine Learning Step 4: G̶r̸i̵d̴S̶e̵a̶r̵c̸h̵C̷V̷

In [122]:
grid_search_cv = GridSearchCV(cv=3, estimator=DecisionTreeClassifier(), param_grid=decision_tree_param_grid)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_estimator_

In [ ]:
abc.fit(X_train, y_train)

In [72]:
mnb.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
rf.fit(X_train, y_train)

In [79]:
sgd.fit(X_train, y_train)

SGDClassifier()

In [104]:
y_pred_abc = abc.predict(X_test)
# y_pred_mnb = mnb.predict(X_test)
# y_pred_rf = rf.predict(X_test)
# y_pred_sgd = sgd.predict(X_test)

### Machine Learning Step Ω: Stratified K-Fold Cross Validation

In [67]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=3)

In [105]:
print(f'{" Classification Report ":=^40}')
print(classification_report(y_test, y_pred_abc, target_names=['Bukan Clickkbait', 'Clickbait']))

Classification Reports
                  precision    recall  f1-score   support

Bukan Clickkbait       0.88      0.86      0.87      1778
       Clickbait       0.86      0.87      0.87      1713

        accuracy                           0.87      3491
       macro avg       0.87      0.87      0.87      3491
    weighted avg       0.87      0.87      0.87      3491



In [106]:

from sklearn.model_selection import StratifiedKFold
  
# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
  
for train_index, test_index in skf.split(X_train, y_train):
    x_train_fold, x_test_fold = X_train, X_test
    y_train_fold, y_test_fold = y_train, y_test
    abc.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(abc.score(x_test_fold, y_test_fold))

KeyboardInterrupt: 